# COMP5046 Assignment2

# 1 - Data Preprocessing

## 1.1Read Data

In [ ]:
import pandas as pd
def read_data(file_name):
    df = pd.read_csv(file_name)
    Sentences = ' '.join(df.Sentence.tolist()).split(' . ')
    word = ' '.join(df.Sentence.tolist()).split(' ')
    try:
        target = ' '.join(df.NER.tolist()).split(' ')

    except:
        target = df.NER.tolist()
    train = [sentence.strip().split() + ['.'] for sentence in Sentences]
    train[-1] = train[-1][:-1]
    i = 0
    tags = []
    for sentence in train:
        tag = target[i:i + len(sentence)]
        tags.append(tag)
        i += len(sentence)
    print("data has been loaded!")
    return train, tags


def read_test_data(file_name):
    df = pd.read_csv(file_name)
    Sentences = ' '.join(df.Sentence.tolist()).split(' . ')
    word = ' '.join(df.Sentence.tolist()).split(' ')
    train = [sentence.strip().split() + ['.'] for sentence in Sentences]
    train[-1] = train[-1][:-1]
    print("test data has been loaded!")
    return train


train_data, target_y_train = read_data("train.csv")
validation_data, target_y_validation = read_data("val.csv")
test_data = read_test_data('test.csv')

## 1.2 Build Features

### 1.2.1 TF-IDF Score

In [ ]:
def tf_idf(data):
    import numpy as np
    DF = {}

    for tokensized_doc in data:
        # get each unique word in the doc - we need to know whether the word is appeared in the document
        for term in np.unique(tokensized_doc):
            try:
                DF[term] += 1
            except:
                DF[term] = 1

    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from collections import Counter
    import math

    tf_idf = {}

    # total number of documents
    N = len(data)

    doc_id = 0
    # get each tokenised doc
    for tokensized_doc in data:
        # initialise counter for the doc
        counter = Counter(tokensized_doc)
        # calculate total number of words in the doc
        total_num_words = len(tokensized_doc)

        # get each unique word in the doc
        for term in np.unique(tokensized_doc):
            # calculate Term Frequency
            tf = counter[term] / total_num_words

            # calculate Document Frequency
            df = DF[term]

            # calculate Inverse Document Frequency
            idf = math.log(N / (df + 1)) + 1

            # calculate TF-IDF
            tf_idf[doc_id, term] = tf * idf

        doc_id += 1

    TF_doc=[]
    for i in range(N):
        temp=[]
        for word in data[i]:
            temp.append(tf_idf[(i,word)])
        TF_doc.append(temp)

    return TF_doc

### 1.2.2 POS Tag

## 1.3 Build Feature to Data

### 1.3.1 Build Word and Tag Dictionary

In [ ]:
word_to_ix = {}
for sentence in train_data+validation_data+test_data:
    for word in sentence:
        word = word.lower()
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in target_y_train+target_y_validation:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

pos_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in POS_tag:
    for tag in tags:
        if tag not in pos_to_ix:
            pos_to_ix[tag] = len(pos_to_ix)

### 1.3.2 Load Pretrained Model

In [ ]:
import gensim.downloader as api
import numpy as np
word_emb_model = api.load("glove-twitter-50")
print("="*89)
print("pre-trained word embedding model has been loaded!")
print("="*89)
EMBEDDING_DIM = 50

embedding_matrix = []
for word in word_list:
    try:
        embedding_matrix.append(word_emb_model.wv[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape

### 1.3.3 Transform Data to Index

In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

In [ ]:
train_pos = POS_tag[:len(train_data)]
val_pos = POS_tag[len(train_data):(len(train_data+validation_data))]
test_pos = POS_tag[-len(test_data):]

train_input_index =  to_index(train_data,word_to_ix)
train_pos_index =  to_index(train_pos,pos_to_ix)
train_output_index = to_index(target_y_train,tag_to_ix)
val_input_index = to_index(validation_data,word_to_ix)
val_pos_index =  to_index(val_pos,pos_to_ix)
val_output_index = to_index(target_y_validation,tag_to_ix)
test_input_index = to_index(test_data,word_to_ix)
test_pos_index =  to_index(test_pos,pos_to_ix)

train_tf_idf_index = tf_idf(train_data)
val_tf_idf_index = tf_idf(validation_data)
test_tf_idf_index = tf_idf(test_data)

# 2 - Model Implementation

## 2.1 Build NER Model

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
           torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))


import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)


def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
           torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))


class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, lstm_dim=90, embedding_dim=50, hidden_dim=128, num_layers=2,
                 method='ATTN_TYPE_DOT_PRODUCT'):
        super(BiLSTM_CRF, self).__init__()
        self.method = method
        self.num_layers = num_layers
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.pos_to_ix = pos_to_ix
        self.posset_size = len(pos_to_ix)
        self.lstm_dim = lstm_dim
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the ,initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        self.lstm = nn.LSTM(lstm_dim, hidden_dim // 2,
                            num_layers=self.num_layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim * 2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2 * self.num_layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2 * self.num_layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence, pos_tagging=None, tf_idf=None):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), -1)
        combined = embeds.unsqueeze(dim=1)
        if pos_tagging is not None:
            pos = torch.eye(self.posset_size).to(device)[pos_tagging].unsqueeze(dim=1)
            combined = torch.cat((combined, pos), 2)
        if tf_idf is not None:
            tf_idf = torch.tensor(np.array(tf_idf), dtype=torch.float).unsqueeze(1).unsqueeze(dim=1)
            combined = torch.cat((combined, tf_idf), 2)
        lstm_out, self.hidden = self.lstm(combined, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)

        return lstm_out

    def _cal_attention(self, lstm_out, method):
        attention_result = torch.zeros(lstm_out.size()[0], self.hidden_dim * 2, device=device)
        if method == 'ATTN_TYPE_DOT_PRODUCT':
            # bmm: https://pytorch.org/docs/master/generated/torch.bmm.html
            for i in range(lstm_out.size()[0]):
                hidden = lstm_out[i]
                attn_weights = F.softmax(torch.bmm(hidden.unsqueeze(0).unsqueeze(0), lstm_out.T.unsqueeze(0)), dim=-1)
                attn_output = torch.bmm(attn_weights, lstm_out.unsqueeze(0))
                concat_output = torch.cat((hidden.unsqueeze(0), attn_output[0]), 1)
                attention_result[i] = concat_output.squeeze(0)
        elif method == 'ATTN_TYPE_SCALE_DOT_PRODUCT':
            for i in range(lstm_out.size()[0]):
                hidden = lstm_out[i]
                attn_weights = F.softmax(
                    1 / np.sqrt(self.hidden_dim) * torch.bmm(hidden.unsqueeze(0).unsqueeze(0), lstm_out.T.unsqueeze(0)),
                    dim=-1)
                attn_output = torch.bmm(attn_weights, lstm_out.unsqueeze(0))
                concat_output = torch.cat((hidden.unsqueeze(0), attn_output[0]), 1)
                attention_result[i] = concat_output.squeeze(0)
        attention_out = self.hidden2tag(attention_result)
        return attention_out

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                    self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags, pos_tagging=None, tf_idf=None):
        lstm_out = self._get_lstm_features(sentence, pos_tagging, tf_idf)

        attention_feats = self._cal_attention(lstm_out, self.method)
        forward_score = self._forward_alg(attention_feats)
        gold_score = self._score_sentence(attention_feats, tags)
        return forward_score - gold_score

    def forward(self, sentence, pos_tagging=None, tf_idf=None):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_out = self._get_lstm_features(sentence, pos_tagging, tf_idf)

        attention_feats = self._cal_attention(lstm_out, self.method)

        # Find the best path, given the features.

        score, tag_seq = self._viterbi_decode(attention_feats)
        return score, tag_seq, attention_feats

## 2.2 Train NER Model

In [ ]:
model_configs = [
    {
        'Embedding': True,
        'Pos': False,
        'tf_idf': False,
        'lstm_dim': 50,
        'accuracy': [],
        'f1 score': []
    },
    {
        'Embedding': True,
        'Pos': True,
        'tf_idf': False,
        'lstm_dim': 89,
        'accuracy': [],
        'f1 score': []
    },
    {
        'Embedding': True,
        'Pos': False,
        'tf_idf': True,
        'lstm_dim': 51,
        'accuracy': [],
        'f1 score': []
    },
    {
        'Embedding': True,
        'Pos': True,
        'tf_idf': True,
        'lstm_dim': 90,
        'accuracy': [],
        'f1 score': []
    }
]

attention_configs = [
    {
        'Embedding': True,
        'Pos': True,
        'tf_idf': True,
        'attention_name': 'ATTN_TYPE_DOT_PRODUCT',
        'accuracy': [],
        'f1 score': []
    },
    {
        'Embedding': True,
        'Pos': True,
        'tf_idf': True,
        'attention_name': 'ATTN_TYPE_SCALE_DOT_PRODUCT',
        'accuracy': [],
        'f1 score': []
    }
]

model_structure_configs = [
    {
        'nlayer': 1,
        'accuracy': [],
        'f1 score': []
    },
    {
        'nlayer': 2,
        'accuracy': [],
        'f1 score': []
    },
    {
        'nlayer': 4,
        'accuracy': [],
        'f1 score': []
    },
    {
        'nlayer': 8,
        'accuracy': [],
        'f1 score': []
    },
]

In [ ]:
def train_iter(model, sentence, target, optimizer, POS_Tagging, tf_idf):
    '''
    Train the model for a single iteration.
    An iteration is when a single batch of data is passed forward and
    backward through the neural network.
    '''
    loss = model.neg_log_likelihood(sentence, target, POS_Tagging, tf_idf)
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
def train_epoch(log_interval, model, device, optimizer, epoch, POS_Tagging=True, tf_idf=True):
    '''
    Train the model for an epoch.
    An epoch is when the entire dataset is passed forward and
    backward through the neural network for once.
    The number of batches in a dataset is equal to number of iterations for one epoch.
    '''
    time1 = datetime.datetime.now()
    train_loss = 0
    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]
        if POS_Tagging:
            pos_index = train_pos_index[i]
            pos_in = torch.tensor(pos_index, dtype=torch.long).to(device)
        else:
            pos_in = None
        if tf_idf:
            tf_idf_index = train_tf_idf_index[i]
            tf_idf_in = torch.tensor(tf_idf_index, dtype=torch.float).to(device)
        else:
            tf_idf_in = None

        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = train_iter(model, sentence=sentence_in, target=targets, optimizer=optimizer, POS_Tagging=pos_in,
                          tf_idf=tf_idf_in)
        train_loss += loss
        time2 = datetime.datetime.now()
        if (i + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.2f}s'.format(
                epoch, (i + 1), len(train_input_index), 100. * (i + 1) / len(train_input_index), train_loss / (i + 1),
                (time2 - time1).total_seconds()))

## 2.3 Save NER Model

# 3 - Evaluate NER Model

In [ ]:
def test(model, input_index, output_index, pos_index=val_pos_index, tf_idf_index=val_tf_idf_index):
    ground_truth = []
    predicted = []
    for i in range(len(input_index)):
        input_sent = input_index[i]
        input_sent = torch.tensor(input_sent, dtype=torch.long).to(device)
        output_tag = output_index[i]
        if pos_index is not None:
            pos_tag = pos_index[i]
            pos_tag = torch.tensor(pos_tag, dtype=torch.long).to(device)
        else:
            pos_tag = None
        if tf_idf_index is not None:
            tf_idf = tf_idf_index[i]
            tf_idf = torch.tensor(tf_idf, dtype=torch.long).to(device)
        else:
            tf_idf = None
        _, prediction, _ = model(input_sent, pos_tag, tf_idf)
        predicted = predicted + prediction
        ground_truth = ground_truth + output_tag
    accuracy = float((np.array(predicted) == np.array(ground_truth)).astype(int).sum()) / float(len(ground_truth))
    f1_ = f1_score(np.array(ground_truth), np.array(predicted),average='micro')
    print('Val accuracy: {:.6f}\tF1 score: {:.2f}'.format(accuracy, f1_),)
    return ground_truth, predicted, accuracy, f1_

## 3.1 Performance Comparison

## 3.2 Ablation Study - different embedding model

In [ ]:
TOTAL_EPOCH = 10
# Different model evaluation
for config in model_configs:
    model = BiLSTM_CRF(vocab_size=len(word_to_ix), lstm_dim=config['lstm_dim'])
    optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
    print("=" * 100)
    print("Start Training")
    for epoch in range(TOTAL_EPOCH):
        train_epoch(500, model=model, device=device, optimizer=optimizer, epoch=epoch, POS_Tagging=config['Pos'],
                    tf_idf=config['tf_idf'])
        print("=" * 100)
        print("Test")
        _, _, accuracy, f1_ = test(model, val_input_index, val_output_index,
                                   pos_index=val_pos_index if config['Pos'] else None,
                                   tf_idf_index=val_tf_idf_index if config['tf_idf'] else None)
        config['accuracy'].append(accuracy)
        print("=" * 100)

## 3.3 Ablation Study - different attention strategy

In [ ]:
for config in attention_configs:
    model = BiLSTM_CRF(vocab_size=len(word_to_ix), num_layers=config['attention_name'])
    optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
    print("=" * 100)
    print("Start Training")
    for epoch in range(TOTAL_EPOCH):
        train_epoch(500, model=model, device=device, optimizer=optimizer, epoch=epoch)
        print("=" * 100)
        print("Test")
        _, _, accuracy, f1_ = test(model, val_input_index, val_output_index)
        config['accuracy'].append(accuracy)
        print("=" * 100)

## 3.4 Ablation Study - different layer strategy

In [ ]:
for config in model_structure_configs:
    model = BiLSTM_CRF(vocab_size=len(word_to_ix), lstm_dim=config['nlayer'])
    optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
    print("=" * 100)
    print("Start Training")
    for epoch in range(TOTAL_EPOCH):
        train_epoch(500, model=model, device=device, optimizer=optimizer, epoch=epoch)
        print("=" * 100)
        print("Test")
        _, _, accuracy, f1_ = test(model, val_input_index, val_output_index)
        config['accuracy'].append(accuracy)
        print("=" * 100)